# Backpropagation with a ROP chain




In [ ]:
%%file /tmp/stack.c
#include <stddef.h>
#include <unistd.h>
#include <sys/mman.h>
#define STACK_SIZE (64 * 1024)
#define PAGE_SIZE  sysconf(_SC_PAGESIZE)

void *original_rsp;
void *stack_top;
/*
// double *dz;   
typedef struct rev_dual {
    double x; // result
    double* dx; // place to put reverse mode gradients.
} rev_dual 

noinline // and maybe this needs a calling convention. Save none?
void returnfromforward(res){
    // restore normal stack
    asm volatile("mov %%rsp, %0" : "=r"(stacktop_rsp));
    asm volatile("mov %0, %%rsp" :: "r"(original_rsp));
    return res;
}

void add(rev_dual x, rev_dual y){
    asm volatile("mov %%rsp, %0" : "=r"(original_rsp));
    asm volatile ("mov %0, %%rsp\n" :: "r"(stack_top));
    dz = alloca(sizeof(double));
    *dz = 0;
    saved = alloca(sizeof(double*));
    saved[0] = dx;
    saved[1] = dy;

    // call next forward. Doesn't return normally

    // maybe I need to put stack swaps in here.
    saved = RETURNFORWARD(x + y, dz);
    // I mean really this is a closure I guess.
    // do backprop
    // hmm. it expects dx and dy to be in registers?
    // hmm by but now saved might be clobbered. Wellllll, maybe if it isn't assumed in a reg it statically knows.
    dx = saved[0];
    dy = saved[1];
    *dx += *dz
    *dy += *dz
    return
}
*/

int main() {
    // you should be doing some checking here for failure, but fuck it. All of this is awful
    void *stack = mmap(NULL, STACK_SIZE + PAGE_SIZE, 
                       PROT_READ | PROT_WRITE,
                       MAP_PRIVATE | MAP_ANONYMOUS, -1, 0);
    mprotect(stack, PAGE_SIZE, PROT_NONE);
     // Stack grows downward, so stack top is at the *end*
    *stack_top = (char *)stack + STACK_SIZE + PAGE_SIZE;

}


Overwriting /tmp/stack.c


In [25]:
! gcc -Wall -O2 -g -Wextra -fanalyzer /tmp/stack.c -o /tmp/stack && /tmp/stack

In [27]:
! objdump -dS  /tmp/stack | grep -A 50 "<main>:"

00000000000010a0 <main>:
    *dx += *dz
    *dy += *dz
}
*/

int main() {
    10a0:	f3 0f 1e fa          	endbr64
    10a4:	53                   	push   %rbx
    // you should be doing some checking here for failure, but fuck it.
    void *stack = mmap(NULL, STACK_SIZE + PAGE_SIZE, 
    10a5:	bf 1e 00 00 00       	mov    $0x1e,%edi
    10aa:	e8 e1 ff ff ff       	call   1090 <sysconf@plt>
    10af:	45 31 c9             	xor    %r9d,%r9d
    10b2:	41 b8 ff ff ff ff    	mov    $0xffffffff,%r8d
    10b8:	31 ff                	xor    %edi,%edi
    10ba:	b9 22 00 00 00       	mov    $0x22,%ecx
    10bf:	48 8d b0 00 00 01 00 	lea    0x10000(%rax),%rsi
    10c6:	ba 03 00 00 00       	mov    $0x3,%edx
    10cb:	e8 a0 ff ff ff       	call   1070 <mmap@plt>
                       PROT_READ | PROT_WRITE,
                       MAP_PRIVATE | MAP_ANONYMOUS, -1, 0);
    mprotect(stack, PAGE_SIZE, PROT_NONE);
    10d0:	bf 1e 00 00 00       	mov    $0x1e,%edi
    void *stack = mmap(NULL, STACK_SIZE + 

Using pytorch graph to achieve other backprop like things.

wengert tapes using ROP. forth interpreter techniques.

https://rufflewind.com/2016-12-30/reverse-mode-automatic-differentiation

https://www.jmlr.org/papers/v18/17-468.html Automatic Differentiation in Machine Learning: a Survey
https://arxiv.org/pdf/1811.05031 A Review of Automatic Differentiation
and its Efficient Implementation
https://www.autodiff.org/

checkpointing
region based memory management

Composing the jacobian vs the transpose jacobian.

We can take little displacements and push them into the codomain.
Or we can take linear functionals and pull them into functionals of the codomain

Connection to exact reals.

Higher Order via chebfun techniques.
Functional integeration via chebfun techniques. chebfun over pytorch values? https://pytorch.org/cppdocs/api/function_namespacetorch_1_1special_1aa8631344adab4a1cf8f70c6920732cba.html
Differentiation of functional integeration via chebfun techniques.

Higher order based on sampling. f(x) + f(1) +... under the assumption the only thing you can do to functions is eval them.
But chebfun enables integ, diff, min, etc.
https://github.com/chebpy/chebpy/blob/master/docs/notebook-implementation-notes.ipynb

Symbolic and automatic aren't _that_ different. it's a difference of shallowness, initial vs final, meta vs object.

http://conal.net/papers/higher-order-ad/higher-order-ad.pdf
https://openreview.net/pdf?id=ryxuz9SzDB the differentiable curry

https://hackage.haskell.org/package/ad-delcont continuations for ad

http://blog.ezyang.com/2019/05/pytorch-internals/ 
https://pytorch.org/blog/overview-of-pytorch-autograd-engine/
https://pytorch.org/blog/computational-graphs-constructed-in-pytorch/

verilog ad



pytroch fx ir https://arxiv.org/abs/2112.08429
https://dl.acm.org/doi/abs/10.1145/3620665.3640366 Pytorch 2: Faster machine learning through dynamic python bytecode transformation and graph compilation

"No. One (torch.jit.trace) produces a (legacy) TorchScript representation while torch.fx.symbolic_trace gives you a shiny new FX one. (But if you ask, maybe torch.compile is the function you actually want…)"

https://pytorch.org/tutorials/intermediate/torch_compile_tutorial.html

triton
https://github.com/gpu-mode/lectures/blob/main/lecture_014/A_Practitioners_Guide_to_Triton.ipynb
https://triton-lang.org/main/index.html
https://openai.com/index/triton/

inductor
https://peps.python.org/pep-0523/ frame evaluator api. torchdynamo


https://github.com/NVIDIA/cutlass cuda templates for linear alg



https://pytorch.org/docs/main/torch.compiler_dynamo_deepdive.html
dynamo is a serious effort to build a reflector in python...
also triton I guess.
also numba


In [7]:
%%file /tmp/compile.py
import torch

@torch.compile
def mse(x, y):
    z = (x - y) ** 2
    return z.sum()

x = torch.randn(200)
y = torch.randn(200)
mse(x, y)


Writing /tmp/compile.py


In [8]:
!TORCH_LOGS=graph_code python /tmp/compile.py

V1226 23:31:40.579000 2317012 torch/fx/passes/runtime_assert.py:114] [0/0] [__graph_code] TRACED GRAPH
V1226 23:31:40.579000 2317012 torch/fx/passes/runtime_assert.py:114] [0/0] [__graph_code]  ===== pre insert_deferred_runtime_asserts __compiled_fn_1 =====
V1226 23:31:40.579000 2317012 torch/fx/passes/runtime_assert.py:114] [0/0] [__graph_code]  <eval_with_key>.0 class GraphModule(torch.nn.Module):
V1226 23:31:40.579000 2317012 torch/fx/passes/runtime_assert.py:114] [0/0] [__graph_code]     def forward(self, L_x_: "f32[200]", L_y_: "f32[200]"):
V1226 23:31:40.579000 2317012 torch/fx/passes/runtime_assert.py:114] [0/0] [__graph_code]         l_x_ = L_x_
V1226 23:31:40.579000 2317012 torch/fx/passes/runtime_assert.py:114] [0/0] [__graph_code]         l_y_ = L_y_
V1226 23:31:40.579000 2317012 torch/fx/passes/runtime_assert.py:114] [0/0] [__graph_code]         
V1226 23:31:40.579000 2317012 torch/fx/passes/runtime_assert.py:114] [0/0] [__graph_code]          # File: /tmp/compile.py:5 in m

In [6]:
import torch
def foo(x, y):
    a = torch.sin(x)
    b = torch.cos(y)
    return a + b
opt_foo1 = torch.compile(foo)
#print(opt_foo1(torch.randn(10, 10), torch.randn(10, 10)))
opt_foo1.get_compiler_config()

{'TYPE_CHECKING': False,
 'enable_auto_functionalized_v2': False,
 'debug': False,
 'disable_progress': True,
 'verbose_progress': False,
 'fx_graph_cache': True,
 'fx_graph_remote_cache': None,
 'autotune_local_cache': True,
 'autotune_remote_cache': None,
 'force_disable_caches': False,
 'sleep_sec_TESTING_ONLY': None,
 'custom_op_default_layout_constraint': 'flexible_layout',
 'cpp_wrapper': False,
 'abi_compatible': False,
 'c_shim_version': '2',
 'dce': False,
 'static_weight_shapes': True,
 'size_asserts': True,
 'nan_asserts': False,
 'pick_loop_orders': True,
 'inplace_buffers': True,
 'allow_buffer_reuse': True,
 'memory_planning': False,
 'memory_pool': 'intermediates',
 'benchmark_harness': True,
 'epilogue_fusion': True,
 'epilogue_fusion_first': False,
 'pattern_matcher': True,
 'b2b_gemm_pass': False,
 'post_grad_custom_pre_pass': None,
 'post_grad_custom_post_pass': None,
 'joint_custom_pre_pass': None,
 'joint_custom_post_pass': None,
 'pre_grad_custom_pass': None,
 '_p

In [32]:
import torch

# Example computation
x = torch.tensor(3.0, requires_grad=True)
y = (x + 2)**2
y
y.grad_fn.next_functions[0]

(<AddBackward0 at 0x781e3eb38dc0>, 0)

In [9]:
def foo(x, y):
    return 2 * x + y

# Run `foo` with the provided inputs and record the tensor operations
traced_foo = torch.jit.trace(foo, (torch.rand(3), torch.rand(3)))
traced_foo.graph


graph(%x : Float(3, strides=[1], requires_grad=0, device=cpu),
      %y : Float(3, strides=[1], requires_grad=0, device=cpu)):
  %2 : Long(requires_grad=0, device=cpu) = prim::Constant[value={2}]() # /tmp/ipykernel_2198523/702319095.py:2:0
  %3 : Float(3, strides=[1], requires_grad=0, device=cpu) = aten::mul(%x, %2) # /tmp/ipykernel_2198523/702319095.py:2:0
  %4 : int = prim::Constant[value=1]() # /tmp/ipykernel_2198523/702319095.py:2:0
  %5 : Float(3, strides=[1], requires_grad=0, device=cpu) = aten::add(%3, %y, %4) # /tmp/ipykernel_2198523/702319095.py:2:0
  return (%5)

In [10]:
traced_foo.code

'def foo(x: Tensor,\n    y: Tensor) -> Tensor:\n  _0 = torch.add(torch.mul(x, CONSTANTS.c0), y)\n  return _0\n'

https://pytorch.org/docs/main/fx.html

In [40]:
for n in traced_foo.graph.nodes():
    print(n)

%2 : Long(requires_grad=0, device=cpu) = prim::Constant[value={2}]() # /tmp/ipykernel_1367663/3114623804.py:2:0

%3 : Float(3, strides=[1], requires_grad=0, device=cpu) = aten::mul(%x, %2) # /tmp/ipykernel_1367663/3114623804.py:2:0

%4 : int = prim::Constant[value=1]() # /tmp/ipykernel_1367663/3114623804.py:2:0

%5 : Float(3, strides=[1], requires_grad=0, device=cpu) = aten::add(%3, %y, %4) # /tmp/ipykernel_1367663/3114623804.py:2:0



Kata:
Z partition function of a string
Action of quantum particle.

In [ ]:
from sympy import *
xn,xs,hbar = symbols('xn xs hbar')

exp(-(x - xn)**2)



In [ ]:
class Expr():
    def __init__(self, value, backward):
        self.value = value
        self.dx = 0
        self.backward = backward
        self.children = children
    def clear():
        self.dx = 0

    def __add__(self, other):
        def back(dx):
            self.dx += dx
            other.dx += dx
        return Expr(self.value + other.value, backward=back, [self,other])




Manual Reverse mode AD
It's always nice to do things manually.
They can be shockingly straightforward.

Put it into 3-address code / ANF / flatten it.






In [4]:
def f(x,y,z):
    q = x + y
    r = q + z
    p = r + y

    def back(dp):
        dr = dp
        dy = dr
        dq = dr
        dz = dr
        dx = dq
        dy += dq
        return dx, dy, dz

    return p, back

res, back = f(1,2,3)
back(4)



(4, 8, 4)

Lenses have multiple forms

- s -> (t, b -> a)
- s -> (t, e) ,  e -> b -> a. closurized form

Also note a functional proraming method of encoding object orietned.
{
    self : e
    getmethod1 : e -> a 
    setmethod1: e -> b -> e
}
So we can consider it to be rearanged as

`s -> {e, getval : e -> t, back : e -> b -> a}` even though persay getval doesn't need e. 


We can do standard tricks like defunctionalization. That is how we can turn it into tape form.

bundle

In [ ]:
class Lens():
    def __init__(self, get, set):
        self.get = get
        self.set = set




Tape.

The tape is kind of keeping a record of evreything you've done.

Mutable cell in python easy choice is list

In [22]:
tape = []
def add(x,y):
    x,dx = x
    y,dy = y
    dz = [0] # a cell
    z = x + y
    tape.append(("add",dz,dx,dy))
    return (z, dz)


# by definition tape will be in 
def backprop():
    while tape:
        op = tape.pop()
        print(op)
        match op:
            case ("add",dz,dx,dy):
                dx[0] += dz[0]
                dy[0] += dz[0]

            case _:
                raise ValueError("unknown op")

def newcell(v):
    return (v, [0])        

x = newcell(1)
y = newcell(2)

res, dres  = add(add(x,y),x)
print(res, dres)
dres[0] = 1
print(dres)

4 [0]
[1]


We can also do multiply.
We can check referential identity during addition to add different things to tape.

This form I think can be egraphed.
Refcells can also _forward_ union find style. Do all equality in the forward pass. Associated 1 refcell per semantic thing.
The indirect reference thing isnt that big a deal in vectorized form.

Noticing "double" is a smart constructor kind of thing. As I showed before, smart constructors are one step away from aegraph style.

hash consing - yes. We don't want to compute x + y twice and then lose sharing.



In [21]:
tape = []
def add(x,y):
    x,dx = x
    y,dy = y
    dz = [0] # a cell
    if dx is dy:
        z = x + y
        tape.append(("double",dz,dx))
    z = x + y
    tape.append(("add",dz,dx,dy))
    return (z, dz)

def mul(x,y):
    x,dx = x
    y,dy = y
    dz = [0] # a cell
    z = x * y
    tape.append(("mul",dz,x,dx,y,dy))
    return (z, dz)

# by definition tape will be in 
def backprop():
    while tape:
        op = tape.pop()
        print(op)
        match op:
            case ("add",dz,dx,dy):
                dx[0] += dz[0]
                dy[0] += dz[0]
            case ("double", dz, dx):
                dx[0] += 2 * dz[0]
            case ("mul",dz,x,dx,y,dy):
                dx[0] += y * dz[0]
                dy[0] += x * dz[0]
            case _:
                raise ValueError("unknown op")

def newcell(v):
    return (v, [0])        

x = newcell(1)
y = newcell(2)

res, dres  = add(add(x,y),x)
print(res, dres)
dres[0] = 1
print(dres)

backprop()
print(x)
print(y)

x = newcell(3)
z = mul(x, mul(x,x))
print(z)
z[1][0] = 1
backprop()
x

4 [0]
[1]
('add', [1], [0], [0])
('add', [1], [1], [0])
(1, [2])
(2, [1])
(27, [0])
('mul', [1], 3, [0], 9, [0])
('mul', [3], 3, [9], 3, [9])


(3, [27])

Do this with a tape stack.
ROP chain

cells are indices into the stack now

In [ ]:
stack = [  add_back, data, data]


def push(n):
    

def ret(n):
    # cleanup and call

add_back():
    x = stack[-1]
    #
    ret(7)

I guess nn.module is already a function object. Like duh.
However, they are hard to integrate / minimize (? I mean, but actuall obviously they are really good). Not great for analysis.

In [ ]:
class OneFun():
    params : pytorch.tensor
    def __call__(self, x):
        


mosiac dex-lang
stalingrad
Higher orderr AD
Julia ran into issues
enzyme
torch-mlir https://github.com/llvm/torch-mlir
remora
Aten https://pytorch.org/cppdocs/notes/tensor_basics.html
pytorch7
cudann
alt backends? dsp cores?
Memory model. No copy
raytracer

https://github.com/huggingface/smol-course
smollm. intreresting. 

# Pytorch


What is necessary and what is not
.pth model files

I had old blog posats using pyhtroch for control and optimzing a wave function.
Yeah, like what is fun?


https://pytorch.org/ecosystem/

executorch

trochtune https://github.com/pytorch/torchtune fine tuning

tensorRT. https://github.com/pytorch/TensorRT
trochao https://github.com/pytorch/ao architecture optimization. qauntization

torch.compile

ONNX deployment https://onnx.ai/
pyro

https://pytorch.org/tutorials/advanced/numpy_extensions_tutorial.html I can call into scipy. If I custom implement backward?
https://pytorch.org/tutorials/intermediate/pinmem_nonblock.html 

https://paperswithcode.com/
https://github.com/yangchris11/samurai 
https://github.com/arplaboratory/learning-to-fly
https://github.com/rl-tools
https://github.com/rl-tools/rl-tools

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="/tmp/data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="/tmp/data",
    train=False,
    download=True,
    transform=ToTensor(),
)
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

device = "cpu"
model = NeuralNetwork().to(device)
print(dir(test_dataloader))
print(model)
dir(model)

# enzyme

https://github.com/EnzymeAD/Enzyme-Tutorial
https://github.com/ftynse/mlir-tutorial-docker/tree/enzyme
tutorial at mlir winter school
enazyme can be on llvm level or on mlir level.
Custom tablegen

```

  grad_C =__enzyme_autodiff((void*)dot,
                    enzyme_const, A,
                    enzyme_dup, B, grad_B,
                    enzyme_out, C,
                    enzyme_const, n);
```
`../dockerscript.sh clang-12 /host/$^ -O3 -Xclang -load -Xclang /Enzyme/enzyme/build/Enzyme/ClangEnzyme-12.so -ffast-math -o /host/$@`

 
jax can output mlir from python?

stableHLO https://github.com/openxla/stablehlo





# triton

https://mastodon.social/@mattpd/115221696829984466 https://github.com/triton-lang/triton/blob/main/python/tutorials/gluon/01-intro.py gluon
https://youtube.com/watch?v=5e1YKqsP8i8&t=1066s https://news.ycombinator.com/item?id=45280592

Blackwell is like a puzzle to get performance?

Tilus https://github.com/NVIDIA/tilus

